In [1]:
import sys
import os
path_pipest = os.path.abspath('./')
n=0
while (not os.path.basename(path_pipest)=='pipest') and (n<6):
    path_pipest=os.path.dirname(path_pipest)
    n+=1 
if not os.path.basename(path_pipest)=='pipest':
    raise ValueError("path_pipest not found. Instead: {}".format(path_pipest))
path_sdhawkes=path_pipest+'/sdhawkes'
path_lobster=path_pipest+'/lobster'
path_lobster_pyscripts=path_lobster+'/py_scripts'
sys.path.append(path_sdhawkes+'/resources/')
sys.path.append(path_sdhawkes+'/modelling/')
sys.path.append(path_lobster_pyscripts+'/')

In [2]:
import numpy as np
from scipy.optimize import minimize as scipy_minimize
from scipy.stats import dirichlet as scipy_dirichlet
from scipy.special import loggamma as LogGamma
from scipy.special import gamma as scipy_gamma

In [3]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [4]:
import simulation

openmp.omp_get_max_threads(): 4


In [5]:
import timeit

In [6]:
def compute_A(alpha,c):
    assert c!=0.0
    bid=np.sum(alpha[1::2])
    ask=np.sum(alpha[0::2])
    diff=bid-ask
    tot=bid+ask
    return (tot-diff/c)/(diff-tot/c)
def compute_AZ(alpha,c):
    bid=np.sum(alpha[1::2])
    ask=np.sum(alpha[0::2])
    A=compute_A(alpha,c)
    Z=(1.0-A)*bid+(1.0+A)*ask
    return A, Z
def produce_candidate(alpha,c):
    A,Z=compute_AZ(alpha,c)
    v=np.zeros((len(alpha),),dtype=np.float)
    v[1::2]=(1.0-A)*alpha[1::2]/Z
    v[0::2]=(1.0+A)*alpha[0::2]/Z
    return v
def logdir(v,alpha): #for the production of M
    return np.sum(alpha*np.log(v))
def produce_M(alpha,l,u):
    v_l=produce_candidate(alpha,l)
    assert np.all(v_l>=0.0)
    v_u=produce_candidate(alpha,u)
    assert np.all(v_u>=0.0)
    return np.exp(max(logdir(v_l,alpha),logdir(v_u,alpha)))
def produce_logM(alpha,l,u,tol=1.0e-10):
    v_l=np.maximum(tol,produce_candidate(alpha,l))
    v_u=np.maximum(tol,produce_candidate(alpha,u))
    return (max(logdir(v_l,alpha),logdir(v_u,alpha)))
def rough_M(alpha):
    S=np.sum(alpha)
    P=np.power(alpha,alpha)
    return np.prod(P)*np.power(S,-S)

In [7]:
def objfun(rho,gamma,l,u):
    alpha=gamma-rho
    def logM():
        return produce_logM(alpha,l,u)
    return logB(rho)+logM()
def logB(rho):
    return np.sum(LogGamma(rho))-LogGamma(np.sum(rho))
def find_rho(gamma,l,u,maxiter=10000, tol=1.0e-8):
    bounds=tuple([(tol,(1.0-tol)*gamma[k]) for k in range(len(gamma))])
    res=scipy_minimize(
        objfun,0.995*gamma,args=(gamma,l,u),
        method='TNC',jac=False,
        bounds=bounds,options={'maxiter': maxiter})
    return res

In [8]:
num_levels, l, u = 2, -0.85, -0.6
gamma=np.random.uniform(low=0.0, high=10.0, size=(2*num_levels,))
avgimb=np.sum(gamma[1::2]-gamma[0::2])/np.sum(gamma)
avgconstr=(l<=avgimb)&(avgimb<u)
res = find_rho(gamma,l,u)
rho = np.array(res['x'],copy=True)
alpha = gamma-rho
M = produce_M(alpha, l, u)
print('l={}; u={}'.format(l,u))
print('gamma: {}'.format(gamma))
print('avg constraint satisfied? {}'.format(avgconstr))
print('target: {}'.format(logB(gamma)))
print('minimisation results:\n {}'.format(res))
print("proposal's improvement: {}".format(res['fun']<logB(gamma)))

l=-0.85; u=-0.6
gamma: [2.63926053 7.15932593 2.83505591 3.01790927]
avg constraint satisfied? False
target: -18.425122875173763
minimisation results:
      fun: -22.98131923340197
     jac: array([6.8428232 , 6.70276279, 5.8339154 , 4.39525465])
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 27
     nit: 4
  status: 2
 success: True
       x: array([2.6392605 , 3.70929691, 2.83505585, 0.61193832])
proposal's improvement: True


In [23]:
num_reject=[]

In [24]:
%%timeit -n 10 -r 10
num_reject.append(
    simulation.naive_sample(num_levels,gamma,l,u))

436 ms ± 132 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [25]:
print('avg num of rejections: {}'
      .format(np.mean(np.array(num_reject))))
print('median num of rejections: {}'
      .format(np.median(np.array(num_reject))))

avg num of rejections: 18378.23
median num of rejections: 12710.0


In [26]:
num_reject=[]

In [27]:
%%timeit -n 10 -r 10
num_reject.append(
    simulation.rejection_sample(num_levels,rho,alpha,l,u,M))

The slowest run took 5.12 times longer than the fastest. This could mean that an intermediate result is being cached.
4.74 ms ± 1.99 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [28]:
print('avg num of rejections: {}'
      .format(np.mean(np.array(num_reject))))
print('median num of rejections: {}'
      .format(np.median(np.array(num_reject))))

avg num of rejections: 178.57
median num of rejections: 121.0
